<h1>Table of Contents<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#Introducción" data-toc-modified-id="Introducción-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>Introducción</a></span></li><li><span><a href="#Exploración-y-transformación-del-csv-previo" data-toc-modified-id="Exploración-y-transformación-del-csv-previo-2"><span class="toc-item-num">2&nbsp;&nbsp;</span>Exploración y transformación del csv previo</a></span></li><li><span><a href="#Actualización-de-datos" data-toc-modified-id="Actualización-de-datos-3"><span class="toc-item-num">3&nbsp;&nbsp;</span>Actualización de datos</a></span></li><li><span><a href="#Chequeamos-los-datos" data-toc-modified-id="Chequeamos-los-datos-4"><span class="toc-item-num">4&nbsp;&nbsp;</span>Chequeamos los datos</a></span></li><li><span><a href="#Creación-de-clases" data-toc-modified-id="Creación-de-clases-5"><span class="toc-item-num">5&nbsp;&nbsp;</span>Creación de clases</a></span></li></ul></div>

# Introducción 

En la sesión de hoy continuaremos con ETL. En concreto aprenderemos como podemos unir datos de distintos orígenes, identificaremos los datos que tenemos y como podremos juntarlos. Además haremos una pequeña limpieza de los datos para que nos quede todo perfecto para seguir nuestro proceso de ETL. 

Como siempre, antes de empezar importamos las librerías que nos harán falta: 

In [1]:
import requests
import pandas as pd
import numpy as np

from datetime import datetime, timedelta

# Exploración y transformación del csv previo

En la lección anterior nos descargamos a información del tiempo para la ciudad de Madrid. Pero la información es de una ciudad concreta para un día concreto. En caso de que no tengáis, tenéis estos datos en la pestaña de datos del gitbook. 

Nosotras lo que en realidad queremos es descargar los datos cada día de la API para tener un seguimiento continuo de los datos, pero hasta el momento solo tenemos los datos para un día.  Lo primero que tendremos que hacer es "traernos" al jupyter el csv que nos guardamos ayer. 

In [2]:
df = pd.read_csv("files/datos_Madrid.csv", index_col = 0)
df.head(2)

,timepoint,cloudcover,lifted_index,prec_type,prec_amount,temp2m,rh2m,weather,wind10m.direction,wind10m.speed
0,3,4,-4,none,0,27,79%,pcloudyday,E,3
1,6,4,-4,none,0,28,80%,pcloudynight,E,3


In [3]:
# vemos que tenemos 64 filas. 
df.shape

(64, 10)

Lo primero que nos damos cuenta es que no tenemos una columna que nos informe del día al que corresponden las medidas, por lo que lo primero que tendremos que hacer es crear una columna con la fecha de las mediciones.

Ya sabemos que Python tiene la librería datetime que nos va a permitir trabajar con fechas. Una de sus funcionalidades es extraer la fecha del día en el que estamos, usando `datetime.now()`. [Aquí](https://docs.python.org/es/3/library/datetime.html) algo de docu. 


Pero claro, tenemos un problema, los datos no son de hoy, son de ayer!! No hay problema, la librería datetime nos permite restar días a nuestro día de hoy usando `timedelta`. Tenéis más info sobre este método en el link anterior. 

In [4]:
ayer = datetime.now() - timedelta(1)
ayer

datetime.datetime(2023, 1, 9, 10, 42, 26, 441942)

Genial! Hemos sacado la fecha de ayer, pero la verdad es que esta un poco feota 😨. De nuevo `datetime` nos permite "formatear" la fecha como nosotras queramos: 

In [5]:
ayer = datetime.strftime(ayer, '%Y-%m-%d')
ayer

'2023-01-09'

Ahora si 💪🏽! Ya tenemos la fecha como queremos. Lo único que tendremos que hacer es crearnos una columna nueva con ese valor: 

In [6]:
df["fecha"] = ayer

In [7]:
# chequeamos si se nos creó esa columna: 

df.head()

,timepoint,cloudcover,lifted_index,prec_type,prec_amount,temp2m,rh2m,weather,wind10m.direction,wind10m.speed,fecha
0,3,4,-4,none,0,27,79%,pcloudyday,E,3,2023-01-09
1,6,4,-4,none,0,28,80%,pcloudynight,E,3,2023-01-09
2,9,4,-4,rain,0,27,78%,ishowernight,E,3,2023-01-09
3,12,3,-4,rain,0,27,77%,ishowernight,E,3,2023-01-09
4,15,2,-4,rain,0,27,80%,ishowernight,NE,3,2023-01-09


De la misma forma que hemos añadido la información de la fecha, vamos a añadir información de la localización de las mediciones. En este caso lo estamos haciendo para Madrid, pero puede que en un futuro queramos incluir otras ciudades. 

En este caso incluiremos las coordenadas y la ciudad. 

In [8]:
df["latitud"] = 40.4165
df["longitud"] = -3.70256
df["ciudad"] = "Madrid"


In [9]:
# chequeamos que todo fue bien

df.head(2)

,timepoint,cloudcover,lifted_index,prec_type,prec_amount,temp2m,rh2m,weather,wind10m.direction,wind10m.speed,fecha,latitud,longitud,ciudad
0,3,4,-4,none,0,27,79%,pcloudyday,E,3,2023-01-09,40.4165,-3.70256,Madrid
1,6,4,-4,none,0,28,80%,pcloudynight,E,3,2023-01-09,40.4165,-3.70256,Madrid


# Actualización de datos 

Perfecto!!! Ya tenemos los datos como queríamos. Ahora lo que tenemos que hacer es "bajarnos" los datos de la API para el día de hoy y juntarlos con los que tenemos de ayer. 

En realidad, el objetivo de este paso es automatizar el proceso de descarga de datos para cada día, por lo que nos crearemos una función. 

La función recibirá tres parámetros:

- `lat`


- `lon` 


- `producto`: hasta ahora solo hemos sacado unos datos en concreto. Pero la API nos puede dar más info y puede que queramos otros datos. 

En cuanto a la latitud y longitud,  hoy queremos los datos de Madrid, pero puede que en algún momento los queramos de otra ciudad. 

Más info sobre los datos de la API [aquí](https://7timer.info/index.php?product=meteo&lon=14.398&lat=47.911&lang=en&ac=0&unit=metric&tzshift=1&site=)

In [10]:
def llamada_API(lat, lon, producto):
    
    # hacemos la llamada  a la API
    url = f'http://www.7timer.info/bin/api.pl?lon=-{lon}&lat={lat}&product={producto}&output=json'

    response = requests.get(url=url)
    codigo_estado = response.status_code
    razon_estado = response.reason
    if codigo_estado == 200:
        print('La peticion se ha realizado correctamente, se ha devuelto el código de estado:',codigo_estado,' y como razón del código de estado: ',razon_estado)
    elif codigo_estado == 402:
        print('No se ha podido autorizar usario, se ha devuelto el código de estado:', codigo_estado,' y como razón del código de estado: ',razon_estado)
    elif codigo_estado == 404:
        print('Algo ha salido mal, el recurso no se ha encontrado,se ha devuelto el código de estado:', codigo_estado,' y como razón del código de estado: ',razon_estado)
    else:
        print('Algo inesperado ha ocurrido, se ha devuelto el código de estado:', codigo_estado,' y como razón del código de estado: ',razon_estado)

    # convertimos los resultados en un dataframe: 
    df = pd.DataFrame.from_dict(pd.json_normalize(response.json()['dataseries']))
    return df

In [11]:
# definimos las variables de las coordenadas y el producto (o tipo de datos) que queremos descargarnos de la API. 

latitud = 40.4165
longitud = -3.70256
producto = "civil"

In [12]:
# nos creamos un dataframe (df_hoy) con los datos de la API para el día de hoy. 

df_hoy = llamada_API(latitud, longitud, producto)
df_hoy.head()

La peticion se ha realizado correctamente, se ha devuelto el código de estado: 200  y como razón del código de estado:  OK


,timepoint,cloudcover,lifted_index,prec_type,prec_amount,temp2m,rh2m,weather,wind10m.direction,wind10m.speed
0,3,1,15,none,0,5,64%,clearnight,NW,3
1,6,2,15,none,0,6,55%,clearnight,NW,3
2,9,1,15,none,0,8,45%,clearday,NW,2
3,12,1,10,none,0,13,43%,clearday,NW,2
4,15,1,10,none,0,13,41%,clearday,N,2


Perfecto! Ya tenemos nuestro *dataset* con la información que necesitamos. Pero recordamos que no tenemos la información de la fecha. Además, el objetivo es ir juntando todos los *dataframes* para tenerlo todo juntito. 

Por lo tanto, nuestro siguientes pasos serían: 

- Incluir la fecha en el *dataframe* nuevo


- Incluir la longitud, latitud y ciudad


- Juntar los *dataframes* 

Nos crearemos una nueva función que recibirá 4 parámetros: 

- El *dataframe* con los datos de hoy. 


- La latitud


- La longitud


- La ciudad

In [13]:
def limpiar_civil(df_nuevo, lat, long, ciudad): 
    
    # lo primero que tenemos que hacer es crear la columna de fecha en el dataframe nuevo 
    # En este caso será la fecha del día de hoy. 
    hoy = datetime.now()
    hoy = datetime.strftime(hoy, '%Y-%m-%d')
    
    # creamos la nueva columna
    df_nuevo["fecha"] = hoy
    
    df_nuevo["latitud"] = lat
    df_nuevo["longitud"] = long
    df_nuevo["ciudad"] = ciudad
    return df_nuevo
    
    

In [14]:
# definimos la ciudad

ciudad = "Madrid"

# recordamos que la latitud y longitud las definimos previamente

print("la latitud es: ", latitud)
print("la longitud es: ", longitud)

la latitud es:  40.4165
la longitud es:  -3.70256


In [15]:
# llamamos a la función

df= limpiar_civil(df_hoy, latitud, longitud, ciudad)
df.head()

,timepoint,cloudcover,lifted_index,prec_type,prec_amount,temp2m,rh2m,weather,wind10m.direction,wind10m.speed,fecha,latitud,longitud,ciudad
0,3,1,15,none,0,5,64%,clearnight,NW,3,2023-01-10,40.4165,-3.70256,Madrid
1,6,2,15,none,0,6,55%,clearnight,NW,3,2023-01-10,40.4165,-3.70256,Madrid
2,9,1,15,none,0,8,45%,clearday,NW,2,2023-01-10,40.4165,-3.70256,Madrid
3,12,1,10,none,0,13,43%,clearday,NW,2,2023-01-10,40.4165,-3.70256,Madrid
4,15,1,10,none,0,13,41%,clearday,N,2,2023-01-10,40.4165,-3.70256,Madrid


Resulta que investigando más la API nos damos cuenta que tiene otros datos. Y vemos que nos interesa sacar los datos sobre la visibilidad del día. Perfecto, manos a la obra para hacer de nuevo la llamada a la API y extraer los datos. 

📌 **NOTA** En este caso la *requests* se hace igual, pero en la url hay un punto donde especificamos el producto. Antes usabamos `civil`, ahora usaremos `astro`.  

💡 Recordamos que al inicio del jupyter nos creamos una función que se llamaba `llamada_API` que podemos reutilizar para extraer los nuevos datos que queremos


In [16]:
# la latitud y la longitud serán las mismas
latitud = 40.4165
longitud = -3.70256

# pero en este caso el producto cambiará. En este caso "astro"
producto2 = "astro"

In [17]:
# llamamamos a la función 

df_astro = llamada_API(latitud, longitud, producto2)

La peticion se ha realizado correctamente, se ha devuelto el código de estado: 200  y como razón del código de estado:  OK


In [18]:
# chequeamos los datos que nos hemos descargado de la API

df_astro.head(2)

,timepoint,cloudcover,seeing,transparency,lifted_index,rh2m,temp2m,prec_type,wind10m.direction,wind10m.speed
0,3,1,5,2,15,8,5,none,NW,3
1,6,2,5,2,15,7,6,none,NW,3


In [19]:
df_astro.shape

(24, 10)

In [20]:
# recordamos como era nuestra información con los datos anteriores

df_hoy.head(2)

,timepoint,cloudcover,lifted_index,prec_type,prec_amount,temp2m,rh2m,weather,wind10m.direction,wind10m.speed,fecha,latitud,longitud,ciudad
0,3,1,15,none,0,5,64%,clearnight,NW,3,2023-01-10,40.4165,-3.70256,Madrid
1,6,2,15,none,0,6,55%,clearnight,NW,3,2023-01-10,40.4165,-3.70256,Madrid


Perfecto, ya tenemos todos los datos. Ahora podemos ver que hay algunas columnas que son comunes entre los *dataframes* pero también que tenemos informacion nueva, en concreto las columnas `seeing` y `transparency`. También nos quedaremos con la columna `timepoint` para poder hacer una correcta unión de los *dataframes* más adelante. Además, vemos que nos falta la columna de fecha. 

Por lo tanto, tenemos que "limpiar" un poco estos datos y juntarlos con la tabla donde lo tenemos todo. 

De nuevo, crearemos una funcioncita 😀 que recibirá como parámetros: 

- el *dataframe* de la nueva llamada a la API. 


- la latitud


- la longitud


- la ciudad

In [21]:
def limpiar_astro(df, lat, long, ciudad):
    
    #seleccionamos solo las columnas que nos interesan
    df = df[["seeing", "transparency", "timepoint"]]
    
    # creamos la columna de fecha: 
    hoy = datetime.now()
    hoy = datetime.strftime(hoy, '%Y-%m-%d')
    df["fecha"] = hoy
    
    
    # insertamos las columnas de la localidad
    df["ciudad"] = ciudad
    
    return df
    
    

In [22]:
# recordamos que la latitud, longitud y ciudad, las definimos previamente

print("la latitud es: ", latitud)
print("la longitud es: ", longitud)
print("la ciudad es: ", ciudad)

la latitud es:  40.4165
la longitud es:  -3.70256
la ciudad es:  Madrid


In [23]:
df_astro = limpiar_astro(df_astro, latitud, longitud, ciudad)
df_astro.head(2)

/tmp/ipykernel_494/3413091675.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df["fecha"] = hoy
/tmp/ipykernel_494/3413091675.py:13: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df["ciudad"] = ciudad


,seeing,transparency,timepoint,fecha,ciudad
0,5,2,3,2023-01-10,Madrid
1,5,2,6,2023-01-10,Madrid


Vale, ya tenemos los datos. Solo nos falta juntar el *dataframe* con datos de otros dias, más los dataframes de los datos que sacamos hoy. En este punto tenemos que tener en cuenta un par de detalles: 

- Tenemos dos tipos de *dataframes*. Dos de ellos con información general, resultado de la llamada  la API de dos dias y el otro con la información de la visibilidad. En este contexto la únión de los *dataframes* será diferente. 

    - Los dataframes con información general los podremos unir usando un `concat` que haremos por las columnas ya que tienen las mismas columnas. 
    
    - En el caso de la información de la visibilidad tendremos que usar un merge ya que tiene  columnas en común, la de fecha. 


- Datos sobre visibilidad no tendremos de ayer, por lo que hacer la unión de los dataframes contendrá nulos. 

Creemonos una nueva función para tener un único *dataframe* con toda la información.  

In [24]:
df_hoy.head(2)

,timepoint,cloudcover,lifted_index,prec_type,prec_amount,temp2m,rh2m,weather,wind10m.direction,wind10m.speed,fecha,latitud,longitud,ciudad
0,3,1,15,none,0,5,64%,clearnight,NW,3,2023-01-10,40.4165,-3.70256,Madrid
1,6,2,15,none,0,6,55%,clearnight,NW,3,2023-01-10,40.4165,-3.70256,Madrid


In [25]:
df_astro.head(2)

,seeing,transparency,timepoint,fecha,ciudad
0,5,2,3,2023-01-10,Madrid
1,5,2,6,2023-01-10,Madrid


In [26]:
df_hoy.shape

(64, 14)

In [27]:
df_astro.shape

(24, 5)

Antes de ponernos a hacer la función exploremos un poco los *dataframes* para entender como tenemos que hacer los mergeos entre la información de la visibilidad y el df_completo. 

Si nos fijamos, en el *dataframe* de visibilidad tenemos menos filas porque se hacen menos medidas a lo largo del día. Por este motivo para hacer la unión de toda la info tendremos que hacerlo por las columnas de `timepoint` y `fecha` y `ciudad`. Además si queremos conservar toda la información tendremos que especificar que queremos toda la información de la tabla completa. Si recordamos en el método de merge teníamos el parámetro `how` con el cual especificamos con que tabla nos queremos quedar. 

In [28]:
def juntar_dfs(df_completo, df_visibilidad):
    
    # ahora es el turno de unir el dataframe con la información de la visibilidad con el completo
    # en este caso el how lo ponemos como right ya que queremos que se quede con toda la info que en la segunda tabla que le pasamos que es la que tiene toda la información
    df = pd.merge(df_completo , df_astro , on=['fecha', "timepoint", "ciudad"], how = "right")

    
    return df


In [29]:
df_final = juntar_dfs(df_hoy, df_astro)

In [30]:
df_final.head()

,timepoint,cloudcover,lifted_index,prec_type,prec_amount,temp2m,rh2m,weather,wind10m.direction,wind10m.speed,fecha,latitud,longitud,ciudad,seeing,transparency
0,3,1,15,none,0,5,64%,clearnight,NW,3,2023-01-10,40.4165,-3.70256,Madrid,5,2
1,6,2,15,none,0,6,55%,clearnight,NW,3,2023-01-10,40.4165,-3.70256,Madrid,5,2
2,9,1,15,none,0,8,45%,clearday,NW,2,2023-01-10,40.4165,-3.70256,Madrid,3,2
3,12,1,10,none,0,13,43%,clearday,NW,2,2023-01-10,40.4165,-3.70256,Madrid,2,2
4,15,1,10,none,0,13,41%,clearday,N,2,2023-01-10,40.4165,-3.70256,Madrid,2,2


# Chequeamos los datos

Hemos creado nuestra primera clase en nuestro proceso de ETL, somos unas auténticas programadoras 💪!
Ahora llega el momento de chequear como están nuestro datos. Si tenemos algún nulo, que tipos de datos tenemos, el nombre de las columnas, etc. 

In [31]:
def chequear_datos(df): 
    
    print("Las columnas son:", "\n")
    print(list(df.columns))
    print("-----------------------------------------")
    
    print("Los tipos de datos que tenemos son:", "\n")
    print(df.dtypes)
    print("-----------------------------------------")
    
    print("El porcentaje de nulos:", "\n")
    print((df.isnull().sum() / df.shape[0]) *  100)

In [32]:
chequear_datos(df_final)

Las columnas son: 

['timepoint', 'cloudcover', 'lifted_index', 'prec_type', 'prec_amount', 'temp2m', 'rh2m', 'weather', 'wind10m.direction', 'wind10m.speed', 'fecha', 'latitud', 'longitud', 'ciudad', 'seeing', 'transparency']
-----------------------------------------
Los tipos de datos que tenemos son: 

timepoint              int64
cloudcover             int64
lifted_index           int64
prec_type             object
prec_amount            int64
temp2m                 int64
rh2m                  object
weather               object
wind10m.direction     object
wind10m.speed          int64
fecha                 object
latitud              float64
longitud             float64
ciudad                object
seeing                 int64
transparency           int64
dtype: object
-----------------------------------------
El porcentaje de nulos: 

timepoint            0.0
cloudcover           0.0
lifted_index         0.0
prec_type            0.0
prec_amount          0.0
temp2m               0

Como vemos, nuestro dataframe no está del todo limpio ya que: 

- La columna de fecha no está en formato `datetime` 

- En la columna `rh2m` tenemos el símbolo `%` que nos puede "molestar" cuando queramos hacer operaciones.

- En este caso no tenemos nulos, pero esta bien anticiparse a este problema. Lo haremos en nuestra próxima función.

Creemos una función para limpiar estas columnas y dejarlas perfectamente limpias. Recibirá dos parámetros: 

- dataframe sobre el que queremos hacer las modificaciones

- lista de columnas que podrían tener nulos

In [33]:
def limpiar_dataframe(df, lista_columnas = []): 
    
    #convertimos la fecha a datetime
    df["fecha"] = pd.to_datetime(df["fecha"])
    
    # reemplazamos los nulos de las columnas por la media
     # lista de columnas en las que queremos reemplazar los nulos
    if len(lista_columnas) != 0:
        df[lista_columnas]=df[lista_columnas].fillna(df.median())
    else:
        pass
    
    
    # quitar % 
    df["rh2m"] = df["rh2m"].replace(r"%", "", regex = True)
    
    # guardamos los datos una vez limpios
    df.to_pickle('files/datos_actualizados.pkl')
    df.to_csv('files/datos_actualizados.csv')
    
    return df

In [34]:
df_final = limpiar_dataframe(df_final)

In [35]:
df_final.head(2)

,timepoint,cloudcover,lifted_index,prec_type,prec_amount,temp2m,rh2m,weather,wind10m.direction,wind10m.speed,fecha,latitud,longitud,ciudad,seeing,transparency
0,3,1,15,none,0,5,64,clearnight,NW,3,2023-01-10,40.4165,-3.70256,Madrid,5,2
1,6,2,15,none,0,6,55,clearnight,NW,3,2023-01-10,40.4165,-3.70256,Madrid,5,2


In [36]:
# chequeamos si tenemos algun nulo
df.isnull().sum()

timepoint            0
cloudcover           0
lifted_index         0
prec_type            0
prec_amount          0
temp2m               0
rh2m                 0
weather              0
wind10m.direction    0
wind10m.speed        0
fecha                0
latitud              0
longitud             0
ciudad               0
dtype: int64

In [37]:
# chequeamos los tipos de datos 

df.dtypes

timepoint              int64
cloudcover             int64
lifted_index           int64
prec_type             object
prec_amount            int64
temp2m                 int64
rh2m                  object
weather               object
wind10m.direction     object
wind10m.speed          int64
fecha                 object
latitud              float64
longitud             float64
ciudad                object
dtype: object

**EJERCICIOS** 

En este ejercicio tendréis que usar la misma API que hemos usado hasta ahora y elegir otro punto de información (hasta ahora hemos usado `civil` y `astro`). Descarga los datos, limpialos y juntalos con los datos de los otros puntos de información que hemos obtenido durante esta lección. 

📌 **NOTA** Al juntar toda la información puede que haya algunas columnas que tengan la misma información. Debemos evitar información duplicada en nuestro *dataframe*, por lo que tendremos que hacer una buena selección de las columnas que queremos conservar.

💪🏽 **BONUS** Puede que algunas de las columnas que tengamos de la nueva llamada a la API tengan una pinta como esta 👇🏽

```python
[{'layer': '950mb', 'rh': 9},
 {'layer': '900mb', 'rh': 11},
 {'layer': '850mb', 'rh': 12},
 {'layer': '800mb', 'rh': 6},
 {'layer': '750mb', 'rh': 4},
 {'layer': '700mb', 'rh': 5},
 {'layer': '650mb', 'rh': 7},
 {'layer': '600mb', 'rh': 15},
 {'layer': '550mb', 'rh': 16},
 {'layer': '500mb', 'rh': 13},
 {'layer': '450mb', 'rh': 9},
 {'layer': '400mb', 'rh': 11},
 {'layer': '350mb', 'rh': 10},
 {'layer': '300mb', 'rh': 10},
 {'layer': '250mb', 'rh': 8},
 {'layer': '200mb', 'rh': -3}]
```

El objetivo de este bonus es separar cada elemento de esa lista de diccionarios en columnas diferentes. 


In [38]:
producto ='meteo'

In [39]:

clima = llamada_API(latitud, longitud, producto)
clima.head()

La peticion se ha realizado correctamente, se ha devuelto el código de estado: 200  y como razón del código de estado:  OK


,timepoint,cloudcover,highcloud,midcloud,lowcloud,rh_profile,wind_profile,temp2m,lifted_index,rh2m,msl_pressure,prec_type,prec_amount,snow_depth,wind10m.direction,wind10m.speed
0,3,1,-9999,-9999,-9999,"[{'layer': '950mb', 'rh': 3}, {'layer': '900mb...","[{'layer': '950mb', 'direction': 210, 'speed':...",5,15,8,1027,none,0,0,305,3
1,6,2,-9999,-9999,-9999,"[{'layer': '950mb', 'rh': 2}, {'layer': '900mb...","[{'layer': '950mb', 'direction': 345, 'speed':...",6,15,7,1028,none,0,0,305,3
2,9,1,-9999,-9999,-9999,"[{'layer': '950mb', 'rh': 2}, {'layer': '900mb...","[{'layer': '950mb', 'direction': 180, 'speed':...",8,15,5,1029,none,0,0,325,2
3,12,1,-9999,-9999,-9999,"[{'layer': '950mb', 'rh': 1}, {'layer': '900mb...","[{'layer': '950mb', 'direction': 270, 'speed':...",13,10,4,1028,none,0,0,335,2
4,15,1,-9999,-9999,-9999,"[{'layer': '950mb', 'rh': 1}, {'layer': '900mb...","[{'layer': '950mb', 'direction': 180, 'speed':...",13,10,4,1026,none,0,0,355,2


In [40]:
dicccionario ={'key':'value', 'key2':'value2'}

In [41]:
# os recomendamos resetear el index del dataframe de los datos climáticos para que no se repitan los nombres de las columnas.

# El primer problema al que nos podemos enfrentar es que si vemos los tipos de las columnas vemos que estas columnas son objetos, es decir, strings, lo que hará que trabajar con ellas sea un poco complicado: 
clima.dtypes

timepoint             int64
cloudcover            int64
highcloud             int64
midcloud              int64
lowcloud              int64
rh_profile           object
wind_profile         object

# en Python tenemos la librería `ast` que nos permite castear un string que dentro tiene diccionarios, o listas o tuplas a su tipo correspondiente. En nuestro caso, lo que conseguiremos es no tener strings sino listas en la columna. Esto lo haremos de la siguiente forma: 

import ast

clima['wind_profile']= clima['wind_profile'].apply(ast.literal_eval)

# una vez que tengamos la columna cambiada, una fantasía de Pandas es que si hago un apply sobre una columna cuyos valores son diccionarios o listas nos va a genererar una columna con los valores de los diccionarios o listas. Donde cada columna será key del diccionario o cada elemento de la lista. 


x = clima['wind_profile'].apply(pd.Series)


# nos creamos un dataframe nuevo con el resultado de la información de una de las columnas separadas por columnas. Esto nos va a devolver un dataframe donde cada fila será una celda del dataframe anterior. 
x = df['rh_profile'].apply(pd.Series) 

# ¿Qué es lo que ocurre cuando hacemos esto?
# Nos ha creado tantas columnas como valores tuvieramos en la lista. Donde columna es, en este caso, un diccionario (porque nuestra lista esta compuesta por distintos diccionarios)

# Ok, hemos conseguido desempaquetar la información de la lista en distintas columnas. Ahora tenemos que despempaquetar la información de los diccionarios en distintas columnas. En este caso, lo que querremos es que las key de los diccionarios sean los nombres de las columnas y los values los valores de las celdas del dataframe. Volveremos a seguir entonces la misma lógica que antes con el apply, pero en este caso necesitamos hacerlo para todo el dataframe (que es x): 

# Por eso empezamos con un for para iterar por cada una de las columnas. 
for i in range(len(x.columns)): 

    # aplicamos el apply,extraemos el valore de la key "layer" y lo almacenamos en una variable que convertimos a string 
    nombre = "rh_" + str(x[i].apply(pd.Series)["layer"][0]) 

    # hacemos lo mismo con una variable que se llame valores para "guardar" los valores de la celda
    valores = list(x[i].apply(pd.Series)["rh"] )

    # usamos el método insert de los dataframes para ir añadiendo esta información a el dataframe con la información del clima. 
    df.insert(i, nombre, valores)

# una vez que hayamos hecho esto para las dos columnas ya podremos hacer el gropuby para después unir toda la información. 

SyntaxError: invalid syntax (1423928522.py, line 6)